# Convert DAILY PIOMAS binaries to netcdf for ease of use

* **Description**: Reads in binaries and creates netcdfs from daily PIOMAS Data
* **Input data**: Piomas binaries
* **Output data**: Netcdf
* **Creator**: Alice DuVivier
* **Date**: October 2022

Data downloaded here:
https://psc.apl.uw.edu/research/projects/arctic-sea-ice-volume-anomaly/data/model_grid

Based on Robbie Mallett's code:
https://github.com/robbiemallett/piomas_bin_reader

In [1]:
import numpy as np
import pandas as pd
import struct
import xarray as xr
import matplotlib.pyplot as plt

## Read Grid

- Downloaded grid.dat from PIOMAS website. Then had to divide it into half for the lat and lon values
- 8640 lines, so divide at line 4320 (Searched for 318.79)

In [2]:
grids = {}

dims = (120,360)

for i in ['lon','lat']:

    grid = np.array(pd.read_csv(f'grids/{i}grid.dat',
                                header=None,
                                delim_whitespace=True))

    flat_grid = grid.ravel()
    
#     if i == 'lon':

    shaped_grid = flat_grid.reshape(dims)
        
#     else:
        
#         shaped_grid = flat_grid.reshape((360,120))
    
    grids[i] = shaped_grid

In [3]:
grids

{'lon': array([[319.5 , 320.5 , 321.5 , ..., 316.5 , 317.5 , 318.5 ],
        [319.5 , 320.5 , 321.5 , ..., 316.5 , 317.5 , 318.5 ],
        [319.5 , 320.5 , 321.5 , ..., 316.5 , 317.5 , 318.5 ],
        ...,
        [319.23, 319.67, 320.12, ..., 317.88, 318.33, 318.77],
        [319.22, 319.66, 320.09, ..., 317.91, 318.34, 318.78],
        [319.21, 319.64, 320.06, ..., 317.94, 318.36, 318.79]]),
 'lat': array([[48.99, 48.99, 48.99, ..., 48.99, 48.99, 48.99],
        [49.59, 49.59, 49.59, ..., 49.59, 49.59, 49.59],
        [50.15, 50.15, 50.15, ..., 50.15, 50.15, 50.15],
        ...,
        [72.44, 72.44, 72.44, ..., 72.44, 72.44, 72.44],
        [72.57, 72.57, 72.58, ..., 72.58, 72.57, 72.57],
        [72.71, 72.71, 72.71, ..., 72.71, 72.71, 72.71]])}

## Define Function to process PIOMAS binaries

- Note that these are no-leap years, so each year has only 365 days

In [4]:
def process_piomas(year):
    
    binary_dir = f'binaries_daily/hiday.H{year}'
    
    ############################################################
    
    # Read File
    
    with open(binary_dir, mode='rb') as file:
    
        fileContent = file.read()
        data = struct.unpack("f" * (len(fileContent)// 4), fileContent)
        
    ############################################################
    
    # Put it in a 3D array
        
        native_data = np.full((365,dims[0],dims[1]),np.nan)

        for day in range(1,366):
            #print(day)
            
            start = (day-1)*(dims[0]*dims[1])
            end = day*(dims[0]*dims[1])
            thickness_list = np.array(data[start:end])
            
            gridded = thickness_list.reshape(dims[0],dims[1])
            native_data[day-1,:,:] = gridded

            #plt.imshow(gridded)
            
            #break
            
          
    ############################################################
        
    # Output to NetCDF4
        
        ds = xr.Dataset( data_vars={'thickness':(['t','x','y'],native_data)},

                         coords =  {'longitude':(['x','y'],grids['lon']),
                                    'latitude':(['x','y'],grids['lat']),
                                    'day':(['t'],np.array(range(1,366)))})
        
        ds.attrs['data_name'] = 'Daily mean Piomas sea ice thickness data'
        
        ds.attrs['description'] = """Sea ice thickness in meters on the native 360x120 grid, 
                                    data produced by University of Washington Polar Science Center"""
        
        ds.attrs['year'] = f"""These data are for the year {year}"""
        
        ds.attrs['citation'] = """When using this data please use the citation: 
                                Zhang, Jinlun and D.A. Rothrock: Modeling global sea 
                                ice with a thickness and enthalpy distribution model 
                                in generalized curvilinear coordinates,
                                Mon. Wea. Rev. 131(5), 681-697, 2003."""
        
        ds.attrs['code to read'] = """  # Example code to read a month of this data 
    
                                        def read_month_of_piomas(year,month): 
    
                                            data_dir = 'output/' 

                                            with xr.open_dataset(f'{data_dir}{year}.nc') as data: 

                                                ds_month = data.where(int(month) == data.month, drop =True) 

                                                return(ds_month)"""
        
        ds.attrs['python author'] = """Alice DuVivier made the file based off Robbie Mallett's python code. If there's a problem with it, 
                                        email him at robbie.mallett.17@ucl.ac.uk"""
                                
        
        

        output_dir = f'output/'
        
        fout = 'piomas_daily_'+str(year)

        ds.to_netcdf(f'{output_dir}{fout}.nc','w')

    return native_data

## Run the processor

In [6]:
for year in range(1979,2022):
    
    x = process_piomas(year)